In [1]:
!pip install -q -U langchain transformers sentence-transformers faiss-cpu torch accelerate bitsandbytes huggingface_hub
!pip install -q ctransformers langchain_community
! pip install flask-ngrok flask
!pip install jq
!pip install llama-cpp-python==0.2.76 --no-cache-dir
!pip install python-dotenv accelerate bitsandbytes
! pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.1/757.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 20.8 MB/s eta 0:00:00
  Installing bui

In [2]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

In [3]:
import json
import textwrap
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from huggingface_hub import login
from langchain.prompts import PromptTemplate

In [4]:
from google.colab import drive
try:
    drive.mount('/content/drive')
except:
    print("Drive already mounted.")

Mounted at /content/drive


In [6]:
print("\n--- Loading and Processing Knowledge Base ---")
loader = JSONLoader(
    file_path='/content/drive/MyDrive/Model&data/diseases.json',
    jq_schema='.',
    text_content=False
)
documents = loader.load()


--- Loading and Processing Knowledge Base ---


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(documents)


In [8]:
docs[0]

Document(metadata={'source': '/content/drive/MyDrive/Model&data/diseases.json', 'seq_num': 1}, page_content='[{"Disease": "Drug Reaction", "Description": "An adverse drug reaction (ADR) is an injury caused by taking medication. ADRs may occur following a single dose or prolonged administration of a drug or result from the combination of two or more drugs.", "symptoms": "rash, itching, hives, redness of skin, swelling of lips, swelling of face, swelling of tongue, fever, chills, shortness of breath, wheezing, cough, chest tightness, dizziness, lightheadedness, fainting, nausea, vomiting, diarrhea, abdominal pain, joint pain, muscle pain, fatigue, headache, blurred vision, jaundice, dark urine, pale stools, peeling skin, blistering skin, mouth ulcers, sore throat, confusion, agitation, irregular heartbeat, low blood pressure, high blood pressure, anaphylaxis", "causes_and_risk_factors": "certain medications, overuse of drugs, allergies to drugs, high doses, multiple drug interactions, ge

In [9]:
print("Loading embedding model...")
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

Loading embedding model...


/tmp/ipython-input-2341024709.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
print("Creating FAISS vector store...")
db = FAISS.from_documents(docs, embeddings)

Creating FAISS vector store...


In [11]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [12]:
from langchain_community.llms import CTransformers

model_path = '/content/drive/MyDrive/Model&data/mistral-7b-instruct-v0.2.Q4_K_M.gguf'
config = {
    'max_new_tokens': 256,
    'temperature': 0.1,
    'context_length': 2048, # Adjust if needed
    'gpu_layers': 50
}
# Load the GGUF model using the CTransformers library
print("Loading GGUF model from Google Drive...")
llm = CTransformers(model=model_path, model_type='mistral', config=config)

print("GGUF Model loaded successfully!")



Loading GGUF model from Google Drive...
GGUF Model loaded successfully!


In [27]:
# --- Create the Prompt ---
# This prompt template ensures the bot behaves correctly
prompt_template = """
### INSTRUCTIONS ###
You are a helpful and polite medical health assistant. Your job is to answer health-related questions accurately based ONLY on the information provided in the context below.
- If the context does not contain enough information to answer the question, you MUST state "I'm sorry, I don't have enough information to answer that question based on my current knowledge base."
- Never invent or assume details outside the given context.
- Keep explanations clear, simple, and non-technical, so anyone can understand.
- Always structure your answer in a clean, easy-to-read format (use short paragraphs, bullet points, or highlights if needed).
- Maintain a relatable and empathetic tone to make the user feel understood.
### CONTEXT ###
{context}

### QUESTION ###
{question}

### HELPFUL ANSWER ###
"""


In [28]:
print("\n--- Setting up the RAG Chain ---")

# Create a PromptTemplate object from the prompt_template string
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt": prompt} # Use the PromptTemplate object here
)


--- Setting up the RAG Chain ---


In [29]:
qa_chain.combine_documents_chain.llm_chain.prompt.template = prompt_template


In [30]:
def ask_question(question):
    result = qa_chain({"query": question})
    print(f"Question: {question}")
    print(f"Answer: {result['result']}\n")

In [31]:
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
import os

NGROK_AUTHTOKEN = "32slYmXPQSoqvFAu8zakxn13ftS_3Rc3Xf9tiWCi2sMbKxPJA"

In [33]:
app = Flask(__name__)
ngrok.set_auth_token(NGROK_AUTHTOKEN)
ngrok.kill()
public_url = ngrok.connect(5000).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

@app.route("/ask", methods=['POST'])


def ask_chatbot():
    data = request.get_json()
    question = data.get("question", "")
    if not question:
        return jsonify({"error": "No question provided"}), 400

    print(f"Received question: {question}")
    result = qa_chain({"query": question})
    print(f"Generated answer: {result['result']}")

    return jsonify({"answer": result['result']})

if __name__ == "__main__":
    app.run(port=5000)

 * ngrok tunnel "https://0b88a547d11f.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
ask_question("I want to know about hypertension")

In [34]:
ask_question("I want to lose my weight. What should I do?")

Question: I want to lose my weight. What should I do?
Answer: Based on the context provided, it seems you're asking about losing weight in general. However, the context doesn't mention any specific health condition related to weight loss. Here are some general tips for maintaining a healthy weight:
1. Balanced Diet: Eat a variety of fruits, vegetables, lean proteins, and whole grains. Limit processed foods, sugars, and saturated fats.
2. Regular Exercise: Aim for at least 30 minutes of moderate-intensity activity most days of the week. This can include walking, cycling, swimming, or even household chores.
3. Stay Hydrated: Drink plenty of water throughout the day to help keep you feeling full and energized.
4. Avoid High Calorie Beverages: Limit your intake of sugary drinks like soda, juice, and alcohol. Opt for water, unsweetened tea, or low-calorie beverages instead.
5. Get Enough Sleep: Aim for 7-9 hours of sleep per night to help regulate your metabolism and maintain energy levels.